In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
from io import BytesIO
import base64
import matplotlib.pyplot as plt

class PersonalFinanceApp:
    def __init__(self):
        self.output = widgets.Output()
        self.create_widgets()

    def create_widgets(self):
        # Title
        self.logo_label = widgets.HTML("<h1 style='text-align:center;'>AI-Powered Personal Finance Manager</h1>")

        # Buttons
        self.track_expenses_button = widgets.Button(description="Track Your Budget", style=widgets.ButtonStyle(button_color='#4C7EFF'))
        self.money_tips_button = widgets.Button(description="See Money Tips", style=widgets.ButtonStyle(button_color='#5AA469'))
        self.predict_spending_button = widgets.Button(description="Smart Recommendations", style=widgets.ButtonStyle(button_color='#8367C7'))
        button_layout = widgets.Layout(width='200px')
        for btn in [self.track_expenses_button, self.money_tips_button, self.predict_spending_button]:
            btn.layout = button_layout

        style = {'description_width' : '100px'}

        # Inputs
        self.expense_text = widgets.HTML("<h3 style='text-align:center;'>Enter your monthly budget below</h3>")
        self.expenses_input = widgets.FloatText(description="Total: ",style = style, layout=widgets.Layout(width='200px'))
        self.rent = widgets.FloatText(description="Rent: ",style = style, layout=widgets.Layout(width='200px'))
        self.utilities = widgets.FloatText(description="Utilities: ",style = style, layout=widgets.Layout(width='200px'))
        self.food = widgets.FloatText(description="Food: ",style = style, layout=widgets.Layout(width='200px'))
        self.transport = widgets.FloatText(description="Transport: ",style = style, layout=widgets.Layout(width='200px'))
        self.healthcare = widgets.FloatText(description="Healthcare: ",style = style, layout=widgets.Layout(width='200px'))
        self.investments = widgets.FloatText(description="Investments: ",style = style , layout=widgets.Layout(width='200px'))
        self.entertainment = widgets.FloatText(description="Entertainment: ",style = style , layout=widgets.Layout(width='200px'))
        self.others = widgets.FloatText(description="Others: ",style = style, layout=widgets.Layout(width='200px'))

        # Stack everything
        self.button_column = widgets.VBox([
            self.logo_label,
            self.expense_text,
            self.expenses_input,
            self.rent,
            self.utilities,
            self.food,
            self.transport,
            self.healthcare,
            self.investments,
            self.entertainment,
            self.others,
            self.track_expenses_button,
            self.money_tips_button,
            self.predict_spending_button
        ], layout=widgets.Layout(align_items='center', justify_content='center', width='100%'))

        display(widgets.VBox([self.button_column, self.output], layout=widgets.Layout(align_items='center', width='100%')))

        # Link buttons once
        self.track_expenses_button.on_click(self.track_expenses)
        self.money_tips_button.on_click(self.see_money_tips)
        self.predict_spending_button.on_click(self.smart_recommendations)

    # --- Track Expenses ---
    def track_expenses(self, b):
        with self.output:
            self.output.clear_output()
            display(HTML("""
        <div style="display:flex; justify-content:center; margin-top:10px;">
            <div style="text-align:left; max-width:600px;">
                <p>----------------------------------------------------------------------------------------------------------------------</p>
            </div>
        </div>
    """))
            try:
                categories = {k: float(v.value or 0) for k,v in [
                    ("Rent", self.rent), ("Utilities", self.utilities), ("Food", self.food),
                    ("Transport", self.transport), ("Healthcare", self.healthcare),
                    ("Investments", self.investments), ("Entertainment", self.entertainment),
                    ("Others", self.others)
                ]}
                total_budget = float(self.expenses_input.value or sum(categories.values()))
                total_spent = sum(categories.values())
                if total_budget <= 0 or total_spent <= 0:
                    display(HTML("<h3>Please enter numbers greater than 0 in all fields.</h3>"))
                    return
                savings = max(0, total_budget - total_spent)

                # Table
                labels = list(categories.keys()) + ["Savings"]
                values = list(categories.values()) + [savings]
                df = pd.DataFrame({"Category": labels, "Amount (QAR)": [f"{v:.2f}" for v in values]})
                html_table = df.to_html(index=False, classes="finance-table")
                display(HTML(f"""
                    <style>
                        .finance-table {{ border: 3px solid #ccc; border-collapse: collapse; text-align: center; margin: auto; }}
                        .finance-table th {{ background-color: #151515; color: white; padding: 8px; }}
                        .finance-table td {{ border: 3px solid #ccc; padding: 8px; }}
                    </style>
                    {html_table}
                """))

                # Pie chart
                colors = ["#E63946", "#F4A261", "#E9C46A", "#2A9D8F", "#457B9D", "#6D597A", "#B56576", "#264653", "#FFBE0B"]
                fig, ax = plt.subplots(figsize=(7,7))
                wedges, texts, autotexts = ax.pie(values, colors=colors, autopct=lambda p: f'{p*sum(values)/100:.0f} QAR',
                                                  startangle=90, wedgeprops={'edgecolor': 'white', 'linewidth': 2, 'antialiased': True},
                                                  textprops={'ha': 'center', 'color': 'white'}, pctdistance=0.8)
                ax.legend(wedges, labels, title="Categories", loc="center left", bbox_to_anchor=(1,0,0.5,1))
                ax.set_title(f"Budget Distribution (Total: {total_budget:.2f} QAR)", color='white', fontsize=16, fontweight='bold')
                ax.axis('equal')
                buffer = BytesIO()
                plt.savefig(buffer, format='png', bbox_inches='tight', transparent=True)
                buffer.seek(0)
                encoded = base64.b64encode(buffer.read()).decode('utf-8')
                plt.close(fig)
                display(HTML(f"<div style='display:flex; justify-content:center;'><img src='data:image/png;base64,{encoded}' width='500'></div>"))
            except ValueError:
                display(HTML("<h3>Please enter valid numbers in all fields.</h3>"))

    # --- Money Tips ---
    def see_money_tips(self, b):
        with self.output:
            self.output.clear_output()
            display(HTML("""
        <div style="display:flex; justify-content:center; margin-top:10px;">
            <div style="text-align:left; max-width:600px;">
                <p>----------------------------------------------------------------------------------------------------------------------</p>
            </div>
        </div>
    """))
            display(HTML("""
                <div style="text-align:left; max-width:600px; margin:auto;">
                    <h3 style="text-align:center;"><b>MONEY TIPS</b></h3>
                    <p>1. <b>50-30-20 Rule</b> → Spend 50% on needs, 30% on wants, 20% on savings/investing.</p>
                    <p>2. <b>Track Every Rial</b> → Log daily expenses to find leaks.</p>
                    <p>3. <b>Emergency Fund</b> → Save 3–6 months of expenses in liquid form.</p>
                    <p>4. <b>Avoid Lifestyle Inflation</b> → Don’t increase expenses when income rises.</p>
                    <p>5. <b>Sleep on Big Purchases</b> → Wait 24 hours before buying non-essentials.</p>
                </div>
            """))

    # --- Smart Recommendations ---
    def smart_recommendations(self, b):
        with self.output:
            self.output.clear_output()
            display(HTML("""
        <div style="display:flex; justify-content:center; margin-top:10px;">
            <div style="text-align:left; max-width:600px;">
                <p>----------------------------------------------------------------------------------------------------------------------</p>
            </div>
        </div>
    """))
            try:
                categories = {k: float(v.value or 0) for k,v in [
                    ("Rent", self.rent), ("Utilities", self.utilities), ("Food", self.food),
                    ("Transport", self.transport), ("Healthcare", self.healthcare),
                    ("Investments", self.investments), ("Entertainment", self.entertainment),
                    ("Others", self.others)
                ]}
                total_budget = float(self.expenses_input.value or sum(categories.values()))
                total_spent = sum(categories.values())
                savings = total_budget - total_spent
                if total_budget <= 0 or total_spent <= 0:
                    display(HTML("<h3>Please enter numbers greater than 0 in all fields.</h3>"))
                    return

                advice = []
                min_savings = total_budget*0.2
                needed_savings = min_savings - savings
                min_values = {"Food": max(50,total_budget*0.05), "Entertainment": max(20,total_budget*0.03),
                              "Transport": max(30,total_budget*0.03), "Others": max(10,total_budget*0.02)}
                discretionary = ["Food","Entertainment","Transport","Others"]

                # Overspending
                if savings < 0:
                  advice.append(f"⚠️ You exceeded budget by {abs(savings):.2f} QAR. Adjust discretionary categories:")
                  for cat in discretionary:
                      min_val = min_values[cat]
                      if categories[cat] > min_val:
                        # Calculate practical reduction
                        reduce_to = max(min_val, categories[cat] + savings*(categories[cat]/total_spent))
                        advice.append(f"   - {cat}: reduce to {reduce_to:.2f} QAR if possible")

                # Need savings
                elif needed_savings > 0:
                    reducible = {cat:max(0,categories[cat]-min_values[cat]) for cat in discretionary}
                    total_reducible = sum(reducible.values())
                    allocated = {}
                    if total_reducible>0:
                        for cat,excess in reducible.items():
                            cut = min(needed_savings*(excess/total_reducible),excess)
                            allocated[cat]=cut
                        advice.append(f"💡 Save {needed_savings:.2f} QAR this month by adjusting:")
                        for cat, cut in allocated.items():
                            advice.append(f"   - {cat}: reduce to {categories[cat]-cut:.2f} QAR")
                    else:
                        advice.append(f"💡 Save {needed_savings:.2f} QAR, but discretionary categories at minimum.")
                else:
                    advice.append("✅ Budget balanced. Savings sufficient. Consider increasing underfunded categories or investments.")

                # Underfunded categories
                for cat in discretionary:
                    if categories[cat]<min_values[cat]:
                        advice.append(f"💡 Increase {cat} to at least {min_values[cat]:.2f} QAR for balance.")

                # Investments
                invest_target = total_budget*0.1
                if categories["Investments"]<invest_target:
                    advice.append(f"💡 Invest at least {invest_target-categories['Investments']:.2f} QAR this month.")

                display(HTML(f"<div style='background:#55555555; padding:15px; border-radius:12px; max-width:600px; margin:auto; text-align:left;'><h3 style='text-align:center;'>Smart Recommendations</h3><p style='font-size:16px; margin:0;'>{'<br>'.join(advice)}</p></div>"))

            except ValueError:
                display(HTML("<h3>Please enter valid numbers in all fields.</h3>"))

# Run the app
finance_app = PersonalFinanceApp()


